## Naive Bayes Model for Costa Rican Poverty Level Predicion

### Outline
**1. Project Setup** \
\
**2. What is a Naive Bayes Model?** \
*2.1 Strengths and Weaknesses of a Naive Bayes model for this project* \
*2.2 Summary of Approach* \
*2.3 Summary of Findings* \
\
**3. Models** \
*3.1 Basic Models* \
*3.2 Improving Model Performance* \
\
**4. Conclusions**

## 1. Project Setup

**Load Data & Packages**

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, recall_score, classification_report, precision_recall_fscore_support, balanced_accuracy_score
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB, ComplementNB
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import VotingClassifier
from imblearn.over_sampling import SMOTE
import os
SEED = 12

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
os.chdir(parent_dir)
scaler = MinMaxScaler()

import load_data as ld
from evaluate_classification import evaluate_classification
df, X_valid, y_valid, train_indices, valid_indices = ld.load_train_data(filepath = 'Kaggle_download/train.csv')

**X and y for training:**

In [ ]:
X = df.iloc[:, :-1]
y = df.loc[:, 'Target']
y_true = y_valid

**Dictionary of Column Classfication by Naive Bayes Model Type**

In [ ]:
# 'rez_esc' not in Gaussian, 'parentesco1' not in bernoulli
nb_datatypes = {'gaussian' :['agesq', 'dependency' , 'edjefa', 'edjefe', 'escolari', 'tamhog', 'max_education_level', 'hh_max_age'], 
                'bernoulli' : ['pisomoscer', 'paredzocalo', 'techoentrepiso', 'elimbasu5', 'tipovivi5', 'tipovivi2', 
                               'instlevel5', 'instlevel3', 'instlevel7', 'coopele', 'planpri', 'v14a', 'parentesco9', 
                               'parentesco10', 'dis', 'estadocivil4', 'parentesco8', 'female', 'eviv1', 'eviv3', 'eviv2', 
                               'estadocivil2', 'parentesco6', 'estadocivil1', 'male', 'estadocivil3', 'mobilephone', 
                               'parentesco7', 'pisonotiene', 'abastaguano', 'parentesco11', 'parentesco12', 'pisonatur', 'pisocemento', 
                               'pisoother', 'pisomadera', 'paredblolad', 'paredfibras', 'paredother', 'paredpreb', 'pareddes', 'paredmad', 
                               'paredzinc', 'techozinc', 'techocane', 'techootro', 'etecho1', 'etecho3', 'etecho2', 'elimbasu2', 'elimbasu3', 
                               'elimbasu1', 'elimbasu4', 'elimbasu6', 'estadocivil5', 'estadocivil7', 'parentesco3', 'parentesco5', 
                               'parentesco2', 'parentesco4', 'cielorazo', 'computer', 'refrig', 'television', 'epared1', 'epared3', 
                               'epared2', 'abastaguadentro', 'abastaguafuera', 'estadocivil6', 'instlevel4', 'instlevel2', 'instlevel6', 
                               'energcocinar2', 'energcocinar3', 'energcocinar4', 'noelec', 'instlevel1', 'energcocinar1', 'sanitario1', 
                               'hacdor', 'hacapo', 'tipovivi1', 'instlevel9', 'tipovivi4', 'lugar4', 'lugar1', 'lugar2', 'lugar5', 'lugar6', 
                               'lugar3', 'tipovivi3', 'sanitario3', 'sanitario5', 'sanitario6', 'sanitario2', 'instlevel8', 'area1', 'v18q', 'area2',"hh_has_marriage"],
                               'multinomial' : ["r4m2", "r4m1", "hhsize", "r4h2", "r4h1", "hogar_nin", "tamviv", "v18q1", "r4t2", "r4t1", 
                                "r4m3", "r4h3", "r4t3", "meaneduc", "qmobilephone", "hogar_total", "overcrowding", "hogar_mayor", 
                                "bedrooms", "hogar_adul"], 'sq_vals' : ['SQBdependency', 'SQBmeaned', 'SQBage', 'SQBhogar_nin', 
                                                                        'SQBhogar_total', 'SQBovercrowding', 'SQBedjefe', 'SQBescolari', "hh_sex_ratio", "hh_child_woman_ratio","hh_child_adult_ratio"]}

## 2. What is a Naive Bayes Model?

The Naive Bayes algorithm assumes that all features are independent of each other, meaning that the presence or absence of one feature does not affect the probability of another feature being present or absent. The algorithm uses Bayes' theorem to calculate the probabilities of different classes given the observed evidence. Bayes' theorem allows us to update our beliefs about the probability of a hypothesis (such as the class of a data point) based on new evidence (such as the features of the data point).

In this case, the Naive Bayes model calculates the probability of a household belonging to 1 of 4 target classes:

- 1 = extreme poverty
- 2 = moderate poverty
- 3 = vulnerable households
- 4 = non vulnerable households

#### 2.1 Strengths and Weaknesses of a Naive Bayes model for this project

##### The benefits of a Naive Bayes model for this project are:
- **Scalability:** Naive bays can handle large datasets and high-dimensional feature spaces efficiently, as the computational complexity of training is linear with the number of features. We have a lot of features in our dataset, and this model is able to parse through many features quickly.

- **Speed:** Naive Bayes models are fast to train and predict. This allows us to run many different versions and experiment with different structures and parameters.

##### The drawbacks of a Naive Bayes model for this project are:
- **Independence assumption:** Naive Bayes models assume that features are conditionally independent given the class label. *This is almost certainly not true with this data.*

- **Data scarcity:** The performance of Naive Bayes models can suffer when there is not enough data to estimate the probabilities accurately. *We know we have a data scarcity problem with this dataset*

- **Continuous features:** Naive Bayes models work better with categorical data. *We have a lot of bianry and continuous data*

- **Imbalanced Datasets**: Naive Bayes models may not perform well on an imbalanced dataset, where the classes are not represented equally. This can lead to poor classification performance, as the Naive Bayes algorithm may be biased towards the majority class.


#### 2.2 Summary of Approach

**There are three main types of Naive Bayes models:**

1. Gaussian Naive Bayes: Gaussian Naive Bayes assumes that the continuous data follows a normal distribution. In our dataset this includes things like age and years of education.

2. Multinomial Naive Bayes: Multinomial Naive Bayes is used for discrete data. In our dataset this includes things like number of people in the household and number of people avode or below a certain age.

3. Bernoulli Naive Bayes: Bernoulli Naive Bayes is also used for discrete data, the presence or absence of a certain attribute or feature. In our dataset this includes all the binary variables such as whether or not the floors are good and whether or not the dwelling has a toilet.

*Each of these models can be run in isolation or in conjunction*

There is also the Complement Naive Bayes Model which is a variation of the standard Naive Bayes algorithm developed to address the issue of imbalanced datasets.


### 2.3 Summary of Findings

*We explored the following models. Accuracy refers to performance on training data.*


| Technique / Model                                                                  | Accuracy |
|------------------------------------------------------------------------------------|----------|
| Standard Gaussian Naive Bayes                                                      | 52%      |
| Gaussian NB on Continuous Features ONLY                                            | 52%      |
| Standard Multinomial Naive Bayes                                                   | 48%      |
| Multinomial NB on Count Features ONLY                                              | 60%      |
| Standard Bernoulli Naive Bayes                                                     | 60%      |
| Bernoulli NB on Binary Features ONLY                                               | 59%      |
| MinMaxScalar on Bernoulli Naive Bayes                                              | 64%      |
| MinMaxScalar on Multinomial Naive Bayes                                            | 19%      |
| MinMaxScalar on Bernoulli Naive Bayes                                              | 60%      |
| Model Ensembling (with MinMaxScalar)                                               | 59%      |
| Complement Naive Bayes                                                             | 51%      |
| Complement NB + MinMaxScaling                                                      | 61%      |
| Oversampling with SMOTE + MinMax Scalar with Multinomial Naive Bayes               | 47%      |


## 3. Models

Each of the models is explained below, with its accompanying accuracy, F1 score, Recall and Confusion Matrix.

### 3.1 Basic Models

#### Basic Gaussian Naive Bayes Model on Full Dataset
Gaussian Naive Bayes: Gaussian Naive Bayes assumes that the continuous data follows a normal distribution. In our dataset this includes things like age and years of education.

In [ ]:
nb = GaussianNB()
nb.fit(X, y)
y_pred = nb.predict(X_valid)
evaluate_classification(y_pred, y_true, cm=True) #evaluate_classification(y_pred, cm = True) to see Confusion Matrix

#### Gaussian Naive Bayes Model on Continuous Features ONLY

In [ ]:
X_g = X[nb_datatypes['gaussian']]
X_valid_g = X_valid[nb_datatypes['gaussian']]
gnb = GaussianNB()
nb.fit(X_g, y)
evaluate_classification(y_pred, y_true, cm=True)

#### Basic Multinomial Naive Bayes on Full Dataset
Multinomial Naive Bayes: Multinomial Naive Bayes is used for discrete data. In our dataset this includes things like number of people in the household and number of people avode or below a certain age.

In [ ]:
nb = MultinomialNB()
nb.fit(X, y)
y_pred = nb.predict(X_valid)
evaluate_classification(y_pred, y_true, cm=True)

#### Multinomial Naive Bayes on Count features ONLY
Multinomial Naive Bayes: Multinomial Naive Bayes is used for discrete data. In our dataset this includes things like number of people in the household and number of people avode or below a certain age.

In [ ]:
X_m = X[nb_datatypes['multinomial']]
X_valid_m = X_valid[nb_datatypes['multinomial']]
gnb = MultinomialNB()
nb.fit(X_m, y)
y_pred = nb.predict(X_valid_m)
evaluate_classification(y_pred, y_true, cm=True)

#### Basic Bernoulli Naive Bayes on Full Dataset
Bernoulli Naive Bayes: Bernoulli Naive Bayes is also used for discrete data, the presence or absence of a certain attribute or feature. In our dataset this includes all the binary variables such as whether or not the floors are good.

In [ ]:
nb = BernoulliNB()
nb.fit(X, y)
y_pred = nb.predict(X_valid)
evaluate_classification(y_pred, y_true, cm=True)

#### Bernoulli Naive Bayes on binary features ONLY

In [ ]:
X_b = X[nb_datatypes['bernoulli']]
X_valid_b = X_valid[nb_datatypes['bernoulli']]
bnb = BernoulliNB()
nb.fit(X_b, y)
y_pred = nb.predict(X_valid_b)
evaluate_classification(y_pred, y_true, cm=True)

### 3.2 Techniques to improve our model

#### MinMaxScalar
 MinMaxScaler is used to scale the numerical features of the dataset. Scaling is a common preprocessing step that can help improve the performance of certain machine learning algorithms, including the Multinomial Naive Bayes.

- Feature scaling: MinMaxScaler scales the numerical features by subtracting the minimum value of the feature and dividing by the range (maximum value - minimum value) for each feature. This ensures that all the features have the same scale.

- Handling non-negative data: MNB assumes that the input features follow a multinomial distribution, which requires non-negative values. By scaling the numerical features using MinMaxScaler, you ensure that all values are non-negative, satisfying the input requirements for MNB.

- Equal weighting: When features have different scales, the algorithm might give more importance to features with larger values. Scaling the features ensures that they all have equal weight in the model, preventing any unintended biases based on the original scale of the data.

**MinMaxScalar on Multinomial**

In [ ]:
X_scaled = scaler.fit_transform(X)

nb = MultinomialNB()
nb.fit(X_scaled, y)
X_valid_scaled = scaler.transform(X_valid)
y_pred = nb.predict(X_valid_scaled)
evaluate_classification(y_pred, y_true, cm=True)


**MinMaxScalar on Gaussian**

In [ ]:
nb = GaussianNB()
nb.fit(X_scaled, y)
X_valid_scaled = scaler.transform(X_valid)
y_pred = nb.predict(X_valid_scaled)
evaluate_classification(y_pred, y_true, cm=True)

**MinMaxScalar on Bernoulli**

In [ ]:
nb = BernoulliNB()
nb.fit(X_scaled, y)
X_valid_scaled = scaler.transform(X_valid)
y_pred = nb.predict(X_valid_scaled)
evaluate_classification(y_pred, y_true, cm=True)

### Model Ensembling (with MinMaxScalar)

More than on Naive Bayes models can be combined when our features are of different data types. This technique is known as "model stacking" or "ensembling." In this approach, we train separate Naive Bayes classifiers for different types of features and then combine their predictions to make a final decision. An ensemble model is implemented below.

In [ ]:
# Scale features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Gaussian features
gaussian_columns = nb_datatypes['gaussian']

# Bernoulli features
bernoulli_columns = nb_datatypes['bernoulli']

# Multinomial features
multinomial_columns = nb_datatypes['multinomial']

# Create Naive Bayes models
gnb = GaussianNB()
bnb = BernoulliNB()
mnb = MultinomialNB()

# Fit models on the respective features
gnb.fit(X.iloc[:, :len(gaussian_columns)], y)
bnb.fit(X.iloc[:, len(gaussian_columns):len(gaussian_columns) + len(bernoulli_columns)], y)
mnb.fit(X.iloc[:, -len(multinomial_columns):], y)

# Create an ensemble model
ensemble_model = VotingClassifier(estimators=[('gnb', gnb), ('bnb', bnb), ('mnb', mnb)], voting='soft')

# Fit the ensemble model
ensemble_model.fit(X, y)

# Make predictions on the validation set
y_pred = ensemble_model.predict(X_valid)
evaluate_classification(y_pred, y_true, cm=True)


### Complement Naive Bayes 

Complement Naive Bayes (CNB) is an extension of Multinomial Naive Bayes that is particularly suited for imbalanced datasets. It is based on the assumption that the complement of a class has a similar distribution to that of the class itself. CNB uses a frequency-based approach, similar to MultinomialNB, but instead of counting the frequency of each feature in each class, it counts the frequency of each feature in the complement of each class.

In order to use CNB with different data types, we can follow a similar approach as with MultinomialNB. First, we need to separate the data into different types, such as numerical, binary, and count data. We can then use a custom transformer to select the relevant features and transform them appropriately. For example, we can use a StandardScaler for numerical data, a BinaryEncoder for binary data, and a CountVectorizer for count data.

Here's an example of how to use CNB with different data types in scikit-learn:

**Simple Complement Naive Bayes**

In [ ]:
cnb = ComplementNB()
cnb.fit(X, y)
y_pred = cnb.predict(X_valid)
evaluate_classification(y_pred, y_true, cm=True)


**Complement Naive Bayes + MinMaxScalar**

In [ ]:
X_scaled = scaler.fit_transform(X)
nb = ComplementNB()
nb.fit(X_scaled, y)
X_valid_scaled = scaler.transform(X_valid)
y_pred = nb.predict(X_valid_scaled)
evaluate_classification(y_pred, y_true, cm = True)

#### Oversampling with SMOTE + MinMax Scalar with Multinomial Naive Bayes

In [ ]:
X_num_scaled = scaler.fit_transform(X)
smote = SMOTE(random_state=10)
X_train_resampled, y_train_resampled = smote.fit_resample(X, y)
nb = MultinomialNB()
nb.fit(X_train_resampled, y_train_resampled)
y_pred = nb.predict(X_valid)
evaluate_classification(y_pred, y_true, cm=True)

## 4. Conclusions

In general, the Naive Bayes Model performed with between 50% and 60% accuracy on training data across various iterations of the model.

Main Takeaways: 
- The model with the best accuracy was the **Bernoulli Naive Bayes with MinMaxScalar**. It had 64% accuracy, and even this model is one of the best performing, its accuracy is primarily due to the fact that it classfies 4, our over represented class, well. The recall values for this model are. Label 1: 0.29 Label 2: 0.38 Label 3: 0.14 Label 4: 0.86
- One of the biggest challenged we face in this project is figuring out how to handle the overrepresenation of group 4 in our data. Models that classfiy most data as 4 (the over represented class) may return higher accuracy but may be overall less useful. 
    - In general, models seemed to behave in 1 of 2 ways:
        1. Classifying most values as 4 (i.e. the *Bernoulli Naive Bayes with MinMaxScalar* above), or
        2. Split values between 2 and 4 (i.e. *Complement Naive Bayes + MinMaxScalar*)
    - **Complement Naive Bayes + MinMaxScalar**: This model has some of the highest accuracy of all the Naive Bayes Models (61%) and is also the best model classfiying 2: Label 2: 0.57 recall and Label 4: 0.79 recall.
- Overall, MinMaxScalar seemed to be an important step to improve accuracy across the board.


# Additional Analysis

In [ ]:
import loops 
df, X_valid, y_valid, train_indices, valid_indices = ld.load_train_data(filepath="Kaggle_download/train.csv", seed=SEED)
scaler = MinMaxScaler()

X_train = df.iloc[:, :-1]
y_train = df.loc[:, 'Target']

### Out of the Box

#### Naive Bayes

In [ ]:
nb = ComplementNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_valid)
evaluate_classification(y_pred, y_valid, cm=False)

### KNN 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_valid)
evaluate_classification(y_pred, y_valid, cm=False)

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state = SEED,
                            n_estimators = 1600,
                            min_samples_split = 2,
                            min_samples_leaf = 1,
                            max_features = 'sqrt',
                            max_depth = 100,
                            bootstrap = False
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_valid)
evaluate_classification(y_pred, y_valid, cm=False)


#### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear', penalty='l2')
lr.fit(X_train, y_train)
y_pred = lr.predict(X_valid)
evaluate_classification(y_pred, y_valid, cm=False)

### Final Model

#### Naive Bayes

In [ ]:
nb = ComplementNB()
loops.loop_model(nb, df, train_indices, valid_indices, scaler=scaler, 
               oversample=ld.gen_SMOTE_data, var_thresh=True)

In [ ]:
nb = BernoulliNB()
loops.loop_model(nb, df, train_indices, valid_indices, scaler=scaler, 
               oversample=ld.gen_SMOTE_data, var_thresh=True)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
nb = KNeighborsClassifier(n_neighbors=15, weights='distance')
loops.loop_model(nb, df, train_indices, valid_indices, scaler=scaler, 
               oversample=ld.gen_SMOTE_data, var_thresh=True)

In [ ]:
# Recode target classes in the DataFrame
df['Target'] = df['Target'].replace({1: 1, 2: 1, 3: 0, 4: 0})

# Recode target classes in the validation data
y_valid['Target'] = y_valid['Target'].replace({1: 1, 2: 1, 3: 0, 4: 0})
nb = ComplementNB()
loops.loop_model(nb, df, train_indices, valid_indices, scaler=scaler, 
               oversample=ld.gen_SMOTE_data, var_thresh=True)

In [ ]:
clf_results = loops.loop_model(clf,df,train_indices,valid_indices)